# Back Testing

This notebook demonstrates several approaches to running back tests: 

* Single Run
* Walk Forward 
* Monte Carlo Simulation

In [ ]:
%use roboquant(3.0.0-SNAPSHOT)
Welcome()

# Single Run
For the back testing examples in this notebook, we'll use a data feed, strategy and metric that come out of the box with _roboquant_.

# Single Run
The most simple back test approach is to run a single back test over all the data available in the feed.


In [ ]:
// The data feed we'll be using 
val feed = AvroFeed.sp25()

// Use the default included Exponential Moving Average Crossover strategy
val strategy = EMACrossover()

// For this notebook we'll only monitor some key Account metrics
val journal = MemoryJournal(AccountMetric())

val account = run(feed, strategy, journal=journal)
account

## Inspect the account

Likely the first thing you want to know after the run has finished is: how does the account look like at the end of the run? 

An account in roboquant is owned by the broker and contains the following 4 types of data: 

1. the cash balances
2. the open positions
3. the open orders


In [ ]:
// You can also get more details, for example about the assets in the portfolio
for ((asset,position) in account.positions) {
    println("${asset.symbol} => ${position.size}")
}

## Inspect metrics
The account contains the state at the end of the run. But metrics are more powerful in that they capture state during each step of the run. So they give an overview how a metric evolves over time during the run. It is the logger who is responsible for storing or logging the metric results.  

In [ ]:
journal.getMetricNames()

In [ ]:
val equity = journal.getMetric("account.equity")
TimeSeriesChart(equity)

# Walk Forward

To move from a single run to a walk forward back test is simple and requires only a few lines of extra code. We run the back test multiple times with each iteration limited to a certain timeframe. 

In [ ]:
feed.timeframe.split(1.years).forEach { 
    val account = run(feed, EMACrossover(), timeframe = it)
    println("timeframe=$it equity=${account.equity()}")
}

# Monte Carlo Simulation

The problem with back testing is that we don't for sure which historic period is most representative for the present. So instead of selecting a set of predefined periods, we'll draw random periods (uniform distributed) from our feed. 

In [ ]:
// Run 25 back-tests each of 6 months
feed.timeframe.sample(6.months, 25).forEach {
    val account = run(feed, EMACrossover(), timeframe = it)
    println("timeframe=$it equity=${account.equity()}")
}